# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
file = "../WeatherPy/output_data/weather_info.csv"

weather_df = pd.read_csv(file)
weather_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Mataura,-46.1927,168.8643,41.49,85,75,2.77,NZ,1650142265
1,1,Jamestown,42.0970,-79.2353,50.52,42,0,14.97,US,1650142265
2,2,Avarua,-21.2078,-159.7750,80.65,83,20,5.75,CK,1650142009
3,3,Tuktoyaktuk,69.4541,-133.0374,-2.20,71,0,5.75,CA,1650142266
4,4,Bengkulu,-3.8004,102.2655,76.89,89,100,3.00,ID,1650142266
...,...,...,...,...,...,...,...,...,...,...
558,558,Chu,43.5983,73.7614,59.34,41,45,7.58,KZ,1650142472
559,559,Suntar,62.1444,117.6319,15.49,93,87,4.92,RU,1650142473
560,560,Rumoi,43.9344,141.6428,39.51,63,90,7.09,JP,1650142473
561,561,Christchurch,-43.5333,172.6333,53.94,84,100,6.91,NZ,1650142256


In [3]:
weather_df.dtypes

Unnamed: 0      int64
City           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
Country        object
Date            int64
dtype: object

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [7]:
# Configure gmaps
# Configure gmaps with API key.
gmaps.configure(api_key=g_key)

# Convert Humidity to float and store
# Also, handle NaN values
weather_df = weather_df.dropna()
humidity = weather_df["Humidity"]

# Store 'Latitude' and 'Longitude' into  locations. 
locations = weather_df[["Lat", "Lng"]]

# Create a humidity Heatmap layer
# Plot Heatmap
fig = gmaps.figure(center = [0,0] ,zoom_level = 2)


# Create and add heat layer 
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                               dissipating=False, max_intensity=100,
                               point_radius = 2)
fig.add_layer(heat_layer)
#Display figure
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
cleaned_weather_df = weather_df.copy()

# max temperature lower than 80 degrees but higher than 70.
cleaned_weather_df = cleaned_weather_df.loc[(cleaned_weather_df["Max Temp"] < 80) &
                                            (cleaned_weather_df["Max Temp"] > 70)]

# Wind speed less than 10 mph
cleaned_weather_df = cleaned_weather_df.loc[cleaned_weather_df["Wind Speed"] < 10]

# Zero cloudiness.
cleaned_weather_df = cleaned_weather_df.loc[cleaned_weather_df["Cloudiness"] == 0]

# Drop rows with null values
cleaned_weather_df = cleaned_weather_df.dropna()

cleaned_weather_df





,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
55,55,Salalah,17.0151,54.0924,79.14,74,0,2.15,OM,1650142284
69,69,Marzuq,14.4000,46.4667,71.55,45,0,5.77,YE,1650142263
99,99,Ribas do Rio Pardo,-20.4431,-53.7592,71.85,51,0,7.90,BR,1650142300
180,180,Parvatsar,26.8833,74.7667,79.47,12,0,4.45,IN,1650142330
238,238,Vao,-22.6667,167.4833,76.46,70,0,8.37,NC,1650142353
244,244,Illapel,-31.6308,-71.1653,71.24,47,0,5.91,CL,1650142355
351,351,Neuquén,-38.9516,-68.0591,71.49,33,0,9.22,AR,1650142395
410,410,Antalaha,-14.9003,50.2788,73.71,85,0,7.14,MG,1650142418


### Hotel Map
* Create a dataframe named `hotel_df` that is the df created above only with the pertinent data.  See the prepared code below (`info_box_template`) to determine what data you will need for your final map.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels (aka "lodging") within 5000 meters.
* Utilize the Google Places API to find the first hotel search result.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [10]:

hotel_df = cleaned_weather_df
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 55: Salalah.
Closest hotel is HAMDAN PLAZA HOTEL SALALAH, an HTG Hotel.
------------
Retrieving Results for Index 69: Marzuq.
Missing field/result... skipping.
------------
Retrieving Results for Index 99: Ribas do Rio Pardo.
Missing field/result... skipping.
------------
Retrieving Results for Index 180: Parvatsar.
Missing field/result... skipping.
------------
Retrieving Results for Index 238: Vao.
Closest hotel is Hotel Kou-Bugny.
------------
Retrieving Results for Index 244: Illapel.
Closest hotel is Hotel Londres.
------------
Retrieving Results for Index 351: Neuquén.
Missing field/result... skipping.
------------
Retrieving Results for Index 410: Antalaha.
Missing field/result... skipping.
------------


In [11]:
hotel_df


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
55,55,Salalah,17.0151,54.0924,79.14,74,0,2.15,OM,1650142284,"HAMDAN PLAZA HOTEL SALALAH, an HTG Hotel"
69,69,Marzuq,14.4000,46.4667,71.55,45,0,5.77,YE,1650142263,NaN
99,99,Ribas do Rio Pardo,-20.4431,-53.7592,71.85,51,0,7.90,BR,1650142300,NaN
180,180,Parvatsar,26.8833,74.7667,79.47,12,0,4.45,IN,1650142330,NaN
238,238,Vao,-22.6667,167.4833,76.46,70,0,8.37,NC,1650142353,Hotel Kou-Bugny
244,244,Illapel,-31.6308,-71.1653,71.24,47,0,5.91,CL,1650142355,Hotel Londres
351,351,Neuquén,-38.9516,-68.0591,71.49,33,0,9.22,AR,1650142395,NaN
410,410,Antalaha,-14.9003,50.2788,73.71,85,0,7.14,MG,1650142418,NaN


In [14]:
# NOTE: Do not change any of the code in this cell
markers = gmaps.marker_layer(locations)

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]


In [15]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=hotel_info
)

# Display figure
fig.add_layer(markers)
fig.add_layer(hotel_layer)
fig




Figure(layout=FigureLayout(height='420px'))